In [2]:
import torch
from transformers import pipeline

#pipe = pipeline("text-generation", model="HuggingFaceH4/starchat-beta", torch_dtype=torch.bfloat16, device_map="auto")
pipe = pipeline("text-generation", model="/data/huggingface/cache/models--HuggingFaceH4--starchat-beta/snapshots/b1bcda690655777373f57ea6614eb095ec2c886f", torch_dtype=torch.bfloat16, device_map="cpu")

OSError: /packages/b1bcda690655777373f57ea6614eb095ec2c886f does not appear to have a file named config.json. Checkout 'https://huggingface.co//packages/b1bcda690655777373f57ea6614eb095ec2c886f/None' for available files.

In [ ]:
pipe("show me the name of flowers")

In [ ]:
# We use a variant of ChatML to format each message
import time

t1 = time.time()

prompt_template = "<|system|>\n<|end|>\n<|user|>\n{query}<|end|>\n<|assistant|>"
prompt = prompt_template.format(query="How do I sort a list in Python?")
print("before model start")
# We use a special <|end|> token with ID 49155 to denote ends of a turn
outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.2, top_k=50, top_p=0.95, eos_token_id=49155)
print("aavesh")
t2 = time.time()
t2-t1

In [ ]:
outputs[0]["generated_text"].split("\n")

In [ ]:
#!pip install transformers==4.34.1

In [ ]:
from refractml import *
from refractml.constants import MLModelFlavours

# # new score functions
from mosaic_utils.ai.score.base import ScoreBase
from typing import Tuple, Union, List, Any
import numpy as np

In [ ]:
class ScoreTemplateExample(ScoreBase):
    """
    This Class Demonstrate How To Implements ScoreBase Interface Class And It Basic Usage.
    """    
    def __init__(self):
        super().__init__()

        self.model_loaded = None

        import torch
        from transformers import pipeline

        if self.model_loaded is None:
            print("LLM model loading from data section")
            self.model_loaded = pipeline("text-generation", model="/data/huggingface/cache/models--HuggingFaceH4--starchat-beta/snapshots/b1bcda690655777373f57ea6614eb095ec2c886f", torch_dtype=torch.bfloat16, device_map="auto")

    def request_processing_fn(self, request) :
    """
    Processes Request Object -> List[Input data, ..]. It could be:
           A List Mapping of All Value Can Be one of : 
               - List[ [Feature_Value1, Feature_Value2, ...], [...] ]
               - List[numpy.array(), numpy.array(), ...]
               - List[tf.Tensor, tf.Tensor, tf.Tensor, ...]
               - List[ SingleSample, SingleSample]

    :return: (n_inputs, payload's)

    Warnings:
    1. Do not reshape your final output for single sample here, do it in prediction.
       Else payloads will be invalidated for extraction at raw and extraction level.
    """
    final_payload = []
    raw_payload = request.json["payload"]
    return (1, raw_payload) 
    
     def pre_processing_fn(self,payload):
        """
         """
        return payload

     def prediction_fn(self,
                       model: Any,
                       input_query 
                       ):
        """
                Does the main prediction on pre_processed_input(Single Sample) using supplied model .

                :param model: Supported Model
                :param pre_processed_input: Single Preprocessed Payload
                :return: Prediction Value From the model
                
                Important Notes:
                - Reshape your data array.reshape(1, -1) before predictions as it contains a single sample.
                    
        """
        prompt_template = "<|system|>\n<|end|>\n<|user|>\n{query}<|end|>\n<|assistant|>"
        text = input_query
        prompt = prompt_template.format(query=text)
#         # We use a special <|end|> token with ID 49155 to denote ends of a turn
        print("promting has started")
        outputs = self.model_loaded(prompt, max_new_tokens=256, do_sample=True, temperature=0.2, top_k=50, top_p=0.95, eos_token_id=49155)
        preds = str(outputs[0]["generated_text"].split("<|assistant|>")[1])
        print("prediction is \n: ",preds)
          return preds
        
#     class Meta:    
         # List of Callables() can be attached For Calling After AnSd Before Scoring
#         def __init__(self):
#             self.name = "Pre Hooked Me !"
#             self.pre_call_hooks.append(self.print_)
#         def print_(self):
#             print(self.name)
#             pre_call_hooks = []
#             post_call_hooks = []

In [ ]:
score_ = ScoreTemplateExample()
import requests
req = requests.Request()

req.json = {"payload":"How do I sort a list in Python?"}

In [ ]:
score_ = ScoreTemplateExample()
import requests
req = requests.Request()

req.json = {"payload":"How do I build neural network in Python?"}

In [ ]:
t3 = time.time()
model_predictions = score_.score(None, req, dry_run=True)
t4 = time.time()
t4-t3

In [ ]:
print(model_predictions)

In [ ]:
model="/data/huggingface/cache/models--HuggingFaceH4--starchat-beta/snapshots/b1bcda690655777373f57ea6614eb095ec2c886f"

In [ ]:
register_model(model,
               ScoreTemplateExample,
               "Starchat_Beta",
               "Starchat_Beta_for_code_Generation",
               MLModelFlavours.pytorch,
               init_script="pip install SentencePiece \\n pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu"
              )